#FIXED WINDOW NEURO LANGUAGE MODEL

*  prefix size = 2 meaning take first two words and predict last
*   we have to tokenize (cut them into substrings) our input and make them into indices numbers




In [ ]:
sentences = ['thanh likes pickles','she always naps', 'victoria is tired','beba likes boba']

In [ ]:
vocab = {} #maps word type to index
inputs = [] #stores an indexified version of each sentence
 
for sent in sentences:
  sent_indx = []
  sent = sent.split() #tokenizing the sentences
  for word in sent:
    if word not in vocab:
      vocab[word] = len(vocab) #add new unique type of index to vocab
    sent_indx.append(vocab[word]) #labeling each word to an index in the sentence
  inputs.append(sent_indx)

print(vocab)
print(inputs)


{'thanh': 0, 'likes': 1, 'pickles': 2, 'she': 3, 'always': 4, 'naps': 5, 'victoria': 6, 'is': 7, 'tired': 8, 'beba': 9, 'boba': 10}
[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 1, 10]]


In [ ]:
import torch


1. convert to LongTensors
2. define inputs and outputs
(given first two words predict third)



In [ ]:
prefixes = torch.LongTensor([sent[:-1] for sent in inputs]) #prefixes first two words 
labels = torch.LongTensor([sent[-1] for sent in inputs]) #last word, defining inputs and outputs

Onto defining the network:


In [ ]:
import torch.nn as nn

class NLM(nn.Module):
  #write init function (initializes all the parameters of the networks)
  #forward function (defines the forward propagation computations)

  def __init__(self, d_embedding, d_hidden, window_size, len_vocab):
    super(NLM, self).__init__() #init the base Module class
    self.embeddings = nn.Embedding(len_vocab, d_embedding)
    self.d_embs = d_embedding
    self.W_hid = nn.Linear(d_embedding*window_size,d_hidden) #concatenated embeddings to hidden
    self.W_out = nn.Linear(d_hidden, len_vocab) #hidden to output probability distribution over vocab

  def forward(self, input): #each input will be a batch of prefixes (for our example is 4)
    batch_size, window_size = input.size() #dimensionality of input, batchsize= 4 sentences, prefix = first 2 words
    embs = self.embeddings(input) #dimension:  4 x 2 x 5
    # print('embedding size:', embs.size())

    #next, we want to concatenate the prefix embeddings together
    concat_embs = embs.view(batch_size, window_size * self.d_embs)
    # print('concatenated embs size:', concat_embs.size())

    #print(embs[0])
    #print(concat_embs[0])

    #now, we project this to the hidden space
    hiddens = self.W_hid(concat_embs)
    # print('hidden size:', hiddens.size())

    #finally, project hiddens to vocabulary space
    outs = self.W_out(hiddens)
    # print('output size:', outs.size()) #unique vocab size is 11

    return outs #return unnormalized probability (logits)





network = NLM(d_embedding=5,d_hidden=12,window_size=2, len_vocab=len(vocab))

num_epochs= 20 #how many times we are going to go through our entire training dataset
learning_rate = 0.1 # modify this if training isn't converging, etc.
loss_fn = nn.CrossEntropyLoss() #average cross entropy loss over each prefix in batch
optimizer = torch.optim.SGD(params=network.parameters(),lr=learning_rate) 

#training loop
for i in range(num_epochs):
  logits = network(prefixes)
  loss = loss_fn(logits, labels)
  
  #update our params to make the loss smaller
  #1. compute gradient (partial derivs of loss with respect to parameters)
  loss.backward()

  #2. update params using gradient descent
  optimizer.step()

  #3. zero out the gradients for the next epoch
  optimizer.zero_grad()
  print('epoch: %d. loss %0.2f' % (i,loss))

epoch: 0. loss 2.72
epoch: 1. loss 2.47
epoch: 2. loss 2.25
epoch: 3. loss 2.04
epoch: 4. loss 1.84
epoch: 5. loss 1.66
epoch: 6. loss 1.48
epoch: 7. loss 1.32
epoch: 8. loss 1.17
epoch: 9. loss 1.03
epoch: 10. loss 0.91
epoch: 11. loss 0.81
epoch: 12. loss 0.72
epoch: 13. loss 0.64
epoch: 14. loss 0.57
epoch: 15. loss 0.51
epoch: 16. loss 0.45
epoch: 17. loss 0.41
epoch: 18. loss 0.37
epoch: 19. loss 0.33


Our loss is decreasing close to zero. But does it predict the right word?

In [ ]:
#let's define a reverse vocabulary mapping (index to word type)
rev_vocab = dict((indx, word) for (word, indx) in vocab.items())

thanhlikes = prefixes[0].unsqueeze(0)
logits = network(thanhlikes)
probs = nn.functional.softmax(logits,dim=1).squeeze()
argmax_indx = torch.argmax(probs).item()
print('given "thanh likes", the model predicts "%s" as the next word with %0.4f probability' % (rev_vocab[argmax_indx], probs[argmax_indx]))

given "thanh likes", the model predicts "pickles" as the next word with 0.7553 probability
